In [1]:
!pip install gdown
!gdown --id 1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn
From (redirected): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn&confirm=t&uuid=b1bb20bb-a2ff-410b-bf89-4b2f2b1be6b4
To: /kaggle/working/data.pkl
100%|███████████████████████████████████████| 1.24G/1.24G [00:08<00:00, 152MB/s]


In [2]:
import pickle
from tqdm import tqdm
import tensorflow as tf
import numpy as np

from preprocess import data_preparation
from preprocess import signal_cutter
from preprocess import data_filtering
from preprocess import resampling
from preprocess import RP_3D
from preprocess import resize_volume
from preprocess import RP_final_data
from preprocess import label_creator
from preprocess import including_data
from models import model_arch
from models import CompModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Read dictionary pkl file
with open('/kaggle/working/data.pkl', 'rb') as fp:
    dataset = pickle.load(fp)

In [4]:
ds = data_preparation(dataset)

# shortening
ds = signal_cutter(ds, d_length=1000)

# denoising
ds = data_filtering(ds)

# resampling
ds = resampling(ds, desired_points = 100)

100%|██████████| 10344/10344 [00:01<00:00, 7691.38it/s]


In [5]:
arr = {'NSR':'426783006', 'AF':'164889003', 'IAVB':'270492004', 'LBBB':'164909002', 'RBBB':'59118001', 'SB':'426177001', 'STach':'427084000'}
labels = label_creator(data_preparation(dataset), arr)
x_train, y_train, x_test, y_test = including_data(labels, ds)
# transformation and resizing
x_train = RP_final_data(x_train)

100%|██████████| 10344/10344 [00:00<00:00, 948470.38it/s]
10344it [00:08, 1219.34it/s]
10344it [00:04, 2157.03it/s]
100%|██████████| 5000/5000 [24:44<00:00,  3.37it/s]


In [6]:
# five-folds preparation

idx_lst = list(range(len(x_train)))

v1 = idx_lst[:1000]
t1 = idx_lst[1000:]

v2 = idx_lst[1000:2000]
t2 = idx_lst[:1000] + idx_lst[2000:]

v3 = idx_lst[2000:3000]
t3 = idx_lst[:2000] + idx_lst[3000:]

v4 = idx_lst[3000:4000]
t4 = idx_lst[:3000] + idx_lst[4000:]

v5 = idx_lst[4000:]
t5 = idx_lst[:4000]

In [7]:
model = model_arch()
model, lr_sch = CompModel(model)

history = model.fit(x_train[t4], y_train.values[t4],
            epochs=100,
            validation_data = (x_train[v4], y_train.values[v4]),
            callbacks = [lr_sch],
            verbose = 1)

Epoch 1/100


2023-11-07 09:31:21.212592: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'Func/gradient_tape/model_3/conv_lstm2d_1/while/model_3/conv_lstm2d_1/while_grad/body/_367/input/_1025' -> 'gradient_tape/model_3/conv_lstm2d_1/while/model_3/conv_lstm2d_1/while_grad/body/_367/gradient_tape/model_3/conv_lstm2d_1/while/gradients/AddN', 'Func/gradient_tape/model_3/conv_lstm2d/while/model_3/conv_lstm2d/while_grad/body/_562/input/_1141' -> 'gradient_tape/model_3/conv_lstm2d/while/model_3/conv_lstm2d/while_grad/body/_562/gradient_tape/model_3/conv_lstm2d/while/gradients/AddN', 'model_3/conv_lstm2d_1/while/body/_179/model_3/conv_lstm2d_1/while/mul_2' -> 'model_3/conv_lstm2d_1/while/body/_179/model_3/conv_lstm2d_1/while/add_5', 'model_3/conv_lstm2d_1/while/body/_179/model_3/conv_lstm2d_1/while/convolution_6' -> 'model_3/conv_lstm2d_1/while/body/_179/model_3/conv_lstm2d_1/while/a

125/125 [==============================] - 207s 1s/step - loss: 0.4375 - acc: 0.3960 - val_loss: 0.4925 - val_acc: 0.2690 - lr: 0.0010
Epoch 2/100
125/125 [==============================] - 179s 1s/step - loss: 0.2883 - acc: 0.6022 - val_loss: 0.3633 - val_acc: 0.4070 - lr: 9.7724e-04
Epoch 3/100
125/125 [==============================] - 179s 1s/step - loss: 0.2532 - acc: 0.6465 - val_loss: 0.3295 - val_acc: 0.5050 - lr: 9.5499e-04
Epoch 4/100
125/125 [==============================] - 179s 1s/step - loss: 0.2299 - acc: 0.6670 - val_loss: 0.2147 - val_acc: 0.7070 - lr: 9.3325e-04
Epoch 5/100
125/125 [==============================] - 179s 1s/step - loss: 0.2133 - acc: 0.6957 - val_loss: 0.2089 - val_acc: 0.6980 - lr: 9.1201e-04
Epoch 6/100
125/125 [==============================] - 179s 1s/step - loss: 0.1950 - acc: 0.7185 - val_loss: 0.1842 - val_acc: 0.7310 - lr: 8.9125e-04
Epoch 7/100
125/125 [==============================] - 179s 1s/step - loss: 0.1805 - acc: 0.7470 - val_loss: 0

In [8]:
model.save('/kaggle/working/RP_ECGNet_Fold4.h5')